# 06 • Protótipo da Lógica do App (sem UI)

Objetivo: validar consulta + paginação + exportação antes de montar a interface Streamlit.


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

# Caminhos do projeto (ajuste se necessário)
BASE_DIR = Path.cwd().resolve().parents[0]  # .../notebooks -> projeto
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"
OUTPUTS_DIR = BASE_DIR / "outputs"

RAW_DIR, PROCESSED_DIR, OUTPUTS_DIR


(WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/raw'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/outputs'))

In [2]:
def normalize_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = s.upper()
    s = re.sub(r"\s+", " ", s)
    return s

MUNICIPIOS_ER = ['Jundiaí', 'Cabreúva', 'Caieiras', 'Cajamar', 'Campo Limpo Paulista', 'Francisco Morato', 'Franco da Rocha', 'Itatiba', 'Itupeva', 'Jarinu', 'Louveira', 'Morungaba', 'Pedra Bela', 'Pinhalzinho', 'Tuiutí', 'Vargem', 'Várzea Paulista', 'Bragança Paulista']
MUNICIPIOS_ER_NORM = set(normalize_text(m) for m in MUNICIPIOS_ER)

sorted(MUNICIPIOS_ER)[:5], len(MUNICIPIOS_ER_NORM)


(['Bragança Paulista',
  'Cabreúva',
  'Caieiras',
  'Cajamar',
  'Campo Limpo Paulista'],
 18)

In [3]:
import sqlite3
from io import BytesIO
import openpyxl

db_path = PROCESSED_DIR / "imoveis_rurais.sqlite"
db_path.exists(), db_path

(True,
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed/imoveis_rurais.sqlite'))

In [4]:
def query_imoveis(municipio_norm, area_min, area_max, titular_like_norm, limit, offset):
    where = []
    params = []

    if municipio_norm:
        where.append("municipio_norm = ?")
        params.append(municipio_norm)

    where.append("area_total_ha BETWEEN ? AND ?")
    params.extend([area_min, area_max])

    if titular_like_norm:
        where.append("titular_norm LIKE ?")
        params.append(f"%{titular_like_norm}%")

    where_sql = ("WHERE " + " AND ".join(where)) if where else ""

    sql = (
        "SELECT codigo_imovel, denominacao, municipio, uf, area_total_ha, titular "
        "FROM imoveis "
        f"{where_sql} "
        "ORDER BY area_total_ha DESC "
        "LIMIT ? OFFSET ?;"
    )
    params.extend([limit, offset])

    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(sql, conn, params=tuple(params))
    conn.close()
    return df

municipio_norm = normalize_text("Jundiaí")
df_res = query_imoveis(municipio_norm, 0, 200, normalize_text("Silva"), limit=50, offset=0)
df_res.head(10)

,codigo_imovel,denominacao,municipio,uf,area_total_ha,titular
0,6290220048551,Sitio Castanho,JUNDIAÍ,SP,4.5000,SILVANA ********************
1,9501491832454,Gleba A1,JUNDIAÍ,SP,2.0085,SILVANA ********************
2,6330545521514,Sitio Bocanera,JUNDIAÍ,SP,1.7000,SILVANO ********************


In [5]:
def df_to_excel_bytes(df: pd.DataFrame) -> bytes:
    output = BytesIO()
    with pd.ExcelWriter(output, engine="openpyxl") as writer:
        df.to_excel(writer, index=False, sheet_name="imoveis")
    return output.getvalue()

excel_bytes = df_to_excel_bytes(df_res)
len(excel_bytes)

5182

Pronto: agora é só ligar isso no Streamlit (sidebar + dataframe + download).